In [1]:
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

In [2]:
data = pd.read_csv(r"tweet_emotions.csv")
data = data[['sentiment', 'content']].dropna()

In [3]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r'\@[\w]+', '', text)
    text = re.sub(r'\#[\w]+', '', text)
    text = re.sub(r'[0-9]+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\s+', ' ', text).strip()
    return text

data['clean_content'] = data['content'].apply(clean_text)

In [4]:
encoder = LabelEncoder()
y = encoder.fit_transform(data['sentiment'])

vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X = vectorizer.fit_transform(data['clean_content'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
model = SVC(kernel='linear', C=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred) * 100)
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=encoder.classes_))


✅ Accuracy: 34.325

📊 Classification Report:
               precision    recall  f1-score   support

       anger       0.00      0.00      0.00        19
     boredom       0.00      0.00      0.00        31
       empty       0.00      0.00      0.00       162
  enthusiasm       0.00      0.00      0.00       163
         fun       0.07      0.01      0.02       338
   happiness       0.32      0.39      0.35      1028
        hate       0.45      0.21      0.28       268
        love       0.50      0.38      0.44       762
     neutral       0.33      0.57      0.42      1740
      relief       0.48      0.03      0.06       352
     sadness       0.35      0.23      0.28      1046
    surprise       0.36      0.04      0.08       425
       worry       0.33      0.44      0.37      1666

    accuracy                           0.34      8000
   macro avg       0.24      0.18      0.18      8000
weighted avg       0.33      0.34      0.31      8000



d:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [6]:
def get_stress_and_depression(emotion):
    high_stress = ['anger', 'fear', 'sadness']
    medium_stress = ['surprise', 'disgust']
    low_stress = ['joy', 'love']
    
    if emotion in high_stress:
        return "High Stress", "Possible Depression Risk"
    elif emotion in medium_stress:
        return "Moderate Stress", "Monitor Mental State"
    else:
        return "Low Stress", "Stable Mental State"


In [7]:
def predict_emotion(text):
    text_clean = clean_text(text)
    text_vec = vectorizer.transform([text_clean])
    emotion_pred = model.predict(text_vec)
    emotion_label = encoder.inverse_transform(emotion_pred)[0]
    
    stress_level, depression_risk = get_stress_and_depression(emotion_label)
    
    print(f"\n Input Text: {text}")
    print(f" Predicted Emotion: {emotion_label}")
    print(f" Stress Level: {stress_level}")
    print(f" Depression Indicator: {depression_risk}")

In [8]:
predict_emotion("I feel really hopeless and sad today.")
predict_emotion("I'm so excited about the new project!")
predict_emotion("I'm angry that no one listened to me.")


 Input Text: I feel really hopeless and sad today.
 Predicted Emotion: sadness
 Stress Level: High Stress
 Depression Indicator: Possible Depression Risk

 Input Text: I'm so excited about the new project!
 Predicted Emotion: happiness
 Stress Level: Low Stress
 Depression Indicator: Stable Mental State

 Input Text: I'm angry that no one listened to me.
 Predicted Emotion: hate
 Stress Level: Low Stress
 Depression Indicator: Stable Mental State
